In [1]:
# materials/stack

import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import zscore

data = pd.read_parquet('data/raw/data_v1.parquet')
forecast = pd.read_parquet('data/processed/data_v1_forecast.parquet')
scores = pd.DataFrame()

In [2]:
forecast

,quarter,quarter_label,ig_maus,ig_revenue,tk_maus,tk_revenue
0,2024-03-31,Q2 2024,2221735781,12735807893,1871336709,4527462823
1,2024-06-30,Q3 2024,2242448858,12929389484,1631989456,3749413402
2,2024-09-30,Q4 2024,2276922459,13251575055,1576689951,3569650540
3,2024-12-31,Q1 2025,2369122780,14113266663,1495697320,3306366674


## Scoring: Last Period

In [3]:
last_results = forecast.tail(1)
ig_last_results = last_results.drop(columns=['tk_maus', 'tk_revenue'])
tk_last_results = last_results.drop(columns=['ig_maus', 'ig_revenue'])

In [ ]:
display(ig_last_results, tk_last_results)

In [ ]:
# Formatar os números com separadores de milhar
lp_ig_revenue = f"{ig_last_results['ig_revenue'].values[0]:,}"
lp_tk_revenue = f"{tk_last_results['tk_revenue'].values[0]:,}"
lp_difference = f"{ig_last_results['ig_revenue'].values[0] - tk_last_results['tk_revenue'].values[0]:,}"

print('Instagram revenues forecast:', lp_ig_revenue)
print('TikTok revenues forecast:', lp_tk_revenue)
print('Difference between Instagram and TikTok:', lp_difference)
print()
if ig_last_results['ig_revenue'].values[0] > tk_last_results['tk_revenue'].values[0]:
    print('Instagram pretended to have more revenues.')
else:
    print('TikTok pretended to have more revenues.')

In [66]:
def calculate_app_scores(data, w1=0.4, w2=0.6):
    """
    Calcula os scores de dois aplicativos baseados em receitas e MAUs.

    Parâmetros:
    - data: DataFrame com colunas 'ig_maus', 'ig_revenue', 'tk_maus', 'tk_revenue'.
    - w1: Peso para a receita (valor entre 0 e 1).
    - w2: Peso para MAUs (valor entre 0 e 1).

    Retorna:
    - DataFrame com os scores dos aplicativos.
    """
    # Normalização
    data['ig_normalized_revenue'] = (data['ig_revenue'] - data['ig_revenue'].min()) / (data['ig_revenue'].max() - data['ig_revenue'].min())
    data['ig_normalized_maus'] = (data['ig_maus'] - data['ig_maus'].min()) / (data['ig_maus'].max() - data['ig_maus'].min())
    data['tk_normalized_revenue'] = (data['tk_revenue'] - data['tk_revenue'].min()) / (data['tk_revenue'].max() - data['tk_revenue'].min())
    data['tk_normalized_maus'] = (data['tk_maus'] - data['tk_maus'].min()) / (data['tk_maus'].max() - data['tk_maus'].min())

    # Cálculo do Score
    data['ig_score'] = w1 * data['ig_normalized_revenue'] + w2 * data['ig_normalized_maus']
    data['tk_score'] = w1 * data['tk_normalized_revenue'] + w2 * data['tk_normalized_maus']

    return data[['quarter', 'ig_score', 'tk_score']]

teste = calculate_app_scores(data)

In [69]:
scoring = data.tail(1)

In [ ]:
scoring

In [ ]:
forecast

In [ ]:
teste

In [ ]:
forecast

In [ ]:
def calculate_scores(df, weight_revenue, weight_maus, x: str, y: str): 
    try:
        # Validar pesos
        # if weight_revenue + weight_maus != 1:
        #    raise ValueError("Os pesos devem somar 1.")

        # Normalizar utilizando Z-score
        df[f'{x}_z'] = zscore(df[f'{x}'])
        df[f'{y}_z'] = zscore(df[f'{y}'])
        

        # Calcular o score composto
        df['score'] = (df[f'{y}_z'] * weight_revenue) + (df[f'{x}_z'] * weight_maus)

        return df
    except Exception as e:
        print(f"Erro ao calcular scores: {e}")

calculate_scores(forecast, 0.5, 0.4, 'ig_maus', 'ig_revenue')

In [ ]:
# Formatar os números com separadores de milhar
lp_ig_maus = f"{ig_last_results['ig_maus'].values[0]:,}"
lp_tk_maus = f"{tk_last_results['tk_maus'].values[0]:,}"
lp_difference = f"{ig_last_results['ig_maus'].values[0] - tk_last_results['tk_maus'].values[0]:,}"

print('Instagram MAUs forecast:', lp_ig_maus)
print('TikTok MAUs forecast:', lp_tk_maus)
print('Difference between Instagram and TikTok:', lp_difference)
print()
if ig_last_results['ig_maus'].values[0] > tk_last_results['tk_maus'].values[0]:
    print('Instagram pretended to have more MAUs.')
else:
    print('TikTok pretended to have more MAUs.')

In [ ]:
# winner score
lp_ig_score = ig_last_results['ig_revenue'].values[0] / ig_last_results['ig_maus'].values[0]
lp_tk_score = tk_last_results['tk_revenue'].values[0] / tk_last_results['tk_maus'].values[0]

print('Instagram score:', lp_ig_score)
print('TikTok score:', lp_tk_score)
print()
if lp_ig_score > lp_tk_score:
    print('Instagram pretended to have a better score.')
else:
    print('TikTok pretended to have a better score.')

## Scoring: All Periods

In [ ]:
display(forecast.drop(columns=['tk_maus', 'tk_revenue']))
display(forecast.drop(columns=['ig_maus', 'ig_revenue']))

In [ ]:
ig_revenue = f"{forecast['ig_revenue'].sum():,}"
tk_revenue = f"{forecast['tk_revenue'].sum():,}"
difference = f"{forecast['ig_revenue'].sum() - forecast['tk_revenue'].sum():,}"

print('Instagram revenues forecast:', ig_revenue)
print('TikTok revenues forecast:', tk_revenue)
print('Difference between Instagram and TikTok:', difference)
print()
if forecast['ig_revenue'].sum() > forecast['tk_revenue'].sum():
    print('Instagram pretended to have more revenues.')
else:
    print('TikTok pretended to have more revenues.')

In [ ]:
ig_maus = f"{forecast['ig_maus'].sum():,}"
tk_maus = f"{forecast['tk_maus'].sum():,}"
difference = f"{forecast['ig_maus'].sum() - forecast['tk_maus'].sum():,}"

print('Instagram MAUs forecast:', ig_maus)
print('TikTok MAUs forecast:', tk_maus)
print('Difference between Instagram and TikTok:', difference)
print()
if forecast['ig_maus'].sum() > forecast['tk_maus'].sum():
    print('Instagram pretended to have more MAUs.')
else:
    print('TikTok pretended to have more MAUs.')

In [ ]:
ig_score = forecast['ig_revenue'].sum() / forecast['ig_maus'].sum()
tk_score = forecast['tk_revenue'].sum() / forecast['tk_maus'].sum()

print('Instagram score:', ig_score)
print('TikTok score:', tk_score)
print()
if ig_score > tk_score:
    print('Instagram pretended to have a better score.')
else:
    print('TikTok pretended to have a better score.')